Importy


In [11]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.output_parser import StrOutputParser
from decouple import config
from langchain_community.llms import Ollama
from langchain.schema import Document
from langchain_experimental.data_anonymizer import PresidioReversibleAnonymizer
from presidio_analyzer import Pattern, PatternRecognizer
from textblob import TextBlob
from pysentimiento import create_analyzer
from accelerate import Accelerator, DataLoaderConfiguration
import re
import pprint

Treść dokumentu do anomizacji wrażliwych dannych, cenzurowania i wykrywania przekleństw i mowy nienawiści 

In [12]:
document_content = """

"""

Stworzenie anonymizera z biblioteki PresidioReversibleAnonymizer służącego do anomizacji dannych oraz ustawienie zmiennej add_default_faker_operators na false ponieważ służy ona do zastąpienia danych zanomizowanych na dane fałszywe

In [13]:

anonymizer = PresidioReversibleAnonymizer(add_default_faker_operators=False,)

Podstawowo anonymizer nie rozpoznaje numerów polskich dowodów i numerów kont bankowych z odstępami jako dane wrażliwe więc dodałem do anonymizer'a oba szablony na podstawie regex'ów.

In [14]:
polish_id_pattern = Pattern(name="polish_id_pattern", regex="[A-Z]{3}\d{6}",score=1)
polish_id_recognizer=PatternRecognizer(supported_entity="POLISH_ID",patterns=[polish_id_pattern])
anonymizer.add_recognizer(polish_id_recognizer)

ssn_pattern = Pattern(name="ssn_pattern", regex="(\d{26}|\d{2} (\d{4} ){5}\d{4})$", score=1)
ssn_recognizer = PatternRecognizer(supported_entity="SSN", patterns=[ssn_pattern])
anonymizer.add_recognizer(ssn_recognizer)

Wywołanie anonymizer'a oraz dzięki funkcji print_colored_pii wyswietlenie dokumentu z odznaczającymi się na czerwono placeholderami

In [15]:
def print_colored_pii(string):
  colored_string = re.sub(r"(<[^>]*>)", lambda m: "\033[31m" + m.group(1) + "\033[0m",string)
  print(colored_string)
  
reversed_anonymize = anonymizer.anonymize(document_content)
print_colored_pii(anonymizer.anonymize(document_content))

Wyświetlenie słownika zmianny dannych


In [16]:
if anonymizer.deanonymizer_mapping:
    print("W dokumencie zostały znaleźone dane wrażliwe")
else:
    print("W dokumencie nie zostały znaleźone dane wrażliwe")
pprint.pprint(anonymizer.deanonymizer_mapping)


W dokumencie nie zostały znaleźone dane wrażliwe
{}


Anomizacja jest również odwraclana


In [17]:
print(anonymizer.deanonymize(reversed_anonymize))

Wykrywanie przekleństw, mowy nienawiści i informacji o stanie zdrowia. Te informacje są wykrywane przez lokalny llm ollama model mistral poprzez wydania polecenia zwrócenia tych danych w cudzysłowie


In [ ]:
ollama = Ollama(model="mistral")

prompt = ChatPromptTemplate.from_template(
    """
    Your task is to detect swear words, hate speech, and names of illnesses in the given text and return only swear 
    words, sentences with hate speech and names of illnesses in single ""  seperate witchout any other comment:
    
    text"{text}
    
    """)

chain = prompt | ollama | StrOutputParser()

response = chain.invoke( {"text":reversed_anonymize})

print(response)

Wyciągnięcie dannych odpowiedzi


In [ ]:
curses = re.findall(r'""(.*?)""', response)
print("ok")
print(curses)

Zastąpienie przekleństw, mowy nienawiści i informacji o zdrowiu Placeholderem


In [20]:
def replace_hate_speech(text, placeholder, curses):
    blob = TextBlob(text)
   
    for word in curses:
        blob = blob.replace(word, placeholder)
    
    return str(blob)
text_with_placeholders = replace_hate_speech(reversed_anonymize,"<HATE_SPEECH_or_illnesses>",curses)

print_colored_pii(text_with_placeholders)

Wykrywanie mowy nienawiści oraz wyznaczenie jej współczynnika (Im wartość bliższa 1 tym tekst zawiera w sobie więcej mowy nienawiści)

In [ ]:
new_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)

accelerator = Accelerator(dataloader_config=new_config)

hatespeech = create_analyzer(task='hate_speech', lang='en')

result = hatespeech.predict(document_content)
print(result.probas.items())


Wykrywanie mowy nienawiści na podstawie sentymentu

In [ ]:
def detect_hate_speech(text):
    blob = TextBlob(text)
    sentiment_score = blob.sentiment.polarity
    if sentiment_score < -0.5:  
        return True
    else:
        return False

if detect_hate_speech(document_content):
    print("Wykryto mowę nienawiści w tekście.")
else:
    print("Nie wykryto mowy nienawiści w tekście.")